In [5]:
import chainer
from chainer import initializers
from chainer import functions

In [4]:
class CapuleLayer(chainer.Link):
    def __init__(self, input_num_capsule, num_capsule,
                 input_dim_vector, dim_vector, num_routing=3):
        super(CapuleLayer, self).__init__()
        self.input_num_capsule = input_num_capsule
        self.num_capsule = num_capsule
        self.input_dim_vector
        self.dim_vector = dim_vector
        self.num_routing = num_routing
        with self.init_scope():
            self.W = chainer.Parameter(initializer=initializers.GlorotUniform(),
                                       (self.input_num_capsule, self.num_capsle,
                                        self.input_dim_vector, self.dim_vector))
            self.bias = chainer.Parameter(initializer=initializers.Zero(),
                                         (1, self.input_num_capsule, self.num_capsule,
                                         1, 1))
    
    def __call__(self, inputs):
        inputs_expand = functions.expand_dims(functions.expand_dims(inputs, 2), 2)
        inputs_tiled = functions.tile(inputs_expand, [1, 1, self.num_capsule, 1, 1])
        inputs_hat = 

SyntaxError: invalid syntax (<ipython-input-4-54813d213eb5>, line 8)